In [12]:
import sys
import os
fname = r'C:\Users\agmontesb\Documents\GitHub\excel\tests\test_base_workbook.py'
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(fname), '..')))

In [13]:
import pytest
import openpyxl as px
import pandas as pd
import itertools
import collections

from excel_workbook import (
    ExcelWorkbook, ExcelTable, 
    cell_address, cell_pattern, 
    data_in_range, tbl_address, rgn_pattern,
    XlErrors, TABLE_DATA_MAP, EMPTY_CELL
    )



In [14]:
def base_workbook(case):
    # Create a base workbook for testing
    filename = r"C:\Users\agmontesb\Downloads\excel_module_test.xlsx"
    wb = px.load_workbook(filename)

    excel_wb = ExcelWorkbook('excel_module_test')

    ws_name = "No links, No parameters"
    ws = wb[ws_name]
    wsheet = excel_wb.create_worksheet(ws_name)

    # Tabla 1
    ws_range = ["G4:I9", "F3:I9"][case]
    fmls, values = data_in_range(ws, ws_range)
    sh1_tbl1 = ExcelTable(wsheet, 'sh1_tbl1', ws_range, fmls, values, recalc=True)
    # m_sh1_tbl1 = sh1_tbl1.minimun_table()

    # Tabla 2
    ws_range = ["G11:H15", "F12:H15"][case]
    fmls, values = data_in_range(ws, ws_range)
    sh1_tbl2 = ExcelTable(wsheet, 'sh1_tbl2', ws_range, fmls, values, recalc=True)
    m_sh1_tbl2 = sh1_tbl2.minimun_table()

    ws_name = "Parameters and inner links"
    ws = wb[ws_name]
    wsheet = excel_wb.create_worksheet(ws_name)

    # Tabla 1
    ws_range = ["F4:H9", "E3:H9"][case]
    fmls, values = data_in_range(ws, ws_range)
    sh2_tbl1 = ExcelTable(wsheet, 'sh2_tbl1', ws_range, fmls, values, recalc=True)
    # m_sh2_tbl1 = sh2_tbl1.minimun_table()

    # Tabla 2
    ws_range = ["F13:H17", "E12:H17"][case]
    fmls, values = data_in_range(ws, ws_range)
    sh2_tbl2 = ExcelTable(wsheet, 'sh2_tbl2', ws_range, fmls, values, recalc=True)
    # m_sh2_tbl2 = sh2_tbl2.minimun_table()

    ws_name = "Outer links, outer parameter"
    ws = wb[ws_name]
    wsheet = excel_wb.create_worksheet(ws_name)

    # Tabla 1
    ws_range = ["F3:H8", "E2:H8"][case]
    fmls, values = data_in_range(ws, ws_range)
    sh3_tbl1 = ExcelTable(wsheet, 'sh3_tbl1', ws_range, fmls, values, recalc=True)
    # m_sh3_tbl1 = sh3_tbl1.minimun_table()

    wb.close()
    return excel_wb


In [15]:
wb = base_workbook(1)

In [16]:
wb.sheets[1]

col,E,F,G,H
row,,,,
2,,,0,
3,tabla 1:,,,
4,l1,700,250,950
5,l2,1200,230,1430
6,l3,340,100,440
7,l4,128,85,213
8,l5,45,18,63
9,,2413,683,3096
10,,,,


In [17]:
ws = wb.sheets[1]
tbl = ws.tables[1]
df = tbl.data

In [18]:
df.loc['G16']

fml           ='sheet1'!A6 + B11
dependents              {B8, B6}
res_order                      1
ftype                          $
value                       2438
code                          B5
Name: G16, dtype: object

In [20]:
field = 'fml'
# values = dict(F13='=+F15+F12', G12='=+E12&F12')   # Fórmula con referencia a celda vacía.
# values = dict(F13='=F5 + G13', F15='=+F14+G14')   # Creación de nuevo enlace externno
# values = dict(H13='=+$G$2*F13 + 75', F17='=+F16+F15+2*F14', G16='=F9 + F13 + F14')  # Cambiar fórmulas en celdas existentes  
# values = dict(F12='=1+2', G12='=G14+F17', H12='=F12+G12')  # Asignar fórmulas en celdas vacías 
# values = dict(F13='=1+2', G14='=F13+G15')  # Asignar fórmulas en celdas con valores

field = 'value'
# values = dict(F13=55, F15=80, G15=22)  # Cambiar valores en celdas existentes  
# values = dict(F12=35, G12=80, H12=22)  # Asignar valores en celdas vacías 
values = dict(G16=333, H13=222)  # Asignar valores en celdas con fórmulas



In [21]:
df1 = tbl.data
set_records(tbl, values, field=field)
df2 = tbl.data

In [11]:
tbl

col,E,F,G,H
row,,,,
12,tabla2:,,tabla2:,
13,l1,15,100,0
14,l2,10,25,35
15,l3,15,38,53
16,l4,80,2428,2508
17,,105,2491,2596


In [ ]:
tbl.get_formula('F12:H17')

In [ ]:
df.dependents.apply(
    lambda x: set(tbl.encoder('decode', code) for code in x) if isinstance(x, set) else ''
    )

In [ ]:
excel_slice = tbl._cell_rgn(tbl.data_rng)
tbl.get_formula(tbl.data_rng)

In [84]:
class TableComparator:

    def __init__(self, df):
        df.loc[:,'dependents'] = df.dependents.apply(lambda items: str(items))
        self.df = df

    def difference(self, other, fields=None):
        x = self.symmetric_difference(other, fields=fields)
        df = x.iloc[range(0, len(x), 2)]
        return df
    
    def symmetric_difference(self, other, fields=None):
        s = self ^ other
        if fields:
            df = s.drop_duplicates(subset=fields, keep=False)
        return df

    def __or__(self, other):
        df = (
            pd.concat([self.df, other.df])
            .drop_duplicates()
            .sort_index()
        )
        return df
    
    def __and__(self, other):
        u = self | other
        mask = u.index.value_counts() == 1
        df = u.loc[mask]
        return df
    
    def __sub__(self, other):
        x = self ^ other
        df = x.iloc[range(0, len(x), 2)]
        return df
    
    def __xor__(self, other):
        u = self | other
        mask = u.index.value_counts() == 2
        df = u.loc[mask]
        return df
    
    def __eq__(self, other):
        df = self ^ other
        return len(df) == 0

In [85]:
P = pd.DataFrame ({"name":["Elizabeth","Darcy"],
        "email":["bennet@xyz.com","darcy@acmecorpus.com"]}).set_index("email")

S = pd.DataFrame ({"name":["Bingley","Elizabeth"],
        "email": ["bingley@xyz.com","bennet@xyz.com"]}).set_index("email")

In [86]:
df1 = TableComparator(df1.df)
df2 = TableComparator(df2.df)

In [94]:
df2.symmetric_difference(df1, ['fml'])

,fml,dependents,res_order,ftype,value,code
cell,,,,,,
F13,=+B15+B25,"{'B2', 'B5'}",1.0,$,15,B12


In [88]:
df2 ^ df1

,fml,dependents,res_order,ftype,value,code
cell,,,,,,
E12,None,None,0.0,$,tabla2:,B22
E12,None,{'B24'},0.0,$,tabla2:,B22
F13,NaN,"{'B2', 'B5'}",0.0,#,25,B12
F13,=+B15+B25,"{'B2', 'B5'}",1.0,$,15,B12
F15,NaN,"{'B4', 'B7'}",0.0,#,15,B15
F15,NaN,"{'B4', 'B12', 'B7'}",0.0,#,15,B15
G16,='sheet1'!A6 + B12,"{'B6', 'B8'}",1.0,$,2438,B5
G16,='sheet1'!A6 + B12,"{'B6', 'B8'}",1.0,$,2428,B5
G17,=B5+B16+B14,nan,2.0,$,2491,B8


In [43]:
df1.df.loc[:,'dependents'] = df1.df.dependents.apply(lambda items: str(items))
df2.df.loc[:, 'dependents'] = df2.df.dependents.apply(lambda items: str(items))
# df1.df

In [60]:
u = pd.concat([df1.df, df2.df]).drop_duplicates().sort_index()

In [63]:
mask = u.index.value_counts() == 1
i = u.loc[mask]

In [70]:
mask = u.index.value_counts() == 2
u.loc[mask].iloc[range(0, len(u.loc[mask]), 2)]

,fml,dependents,res_order,ftype,value,code
cell,,,,,,
E12,None,{'B24'},0.0,$,tabla2:,B22
F13,=+B15+B25,"{'B2', 'B5'}",1.0,$,15,B12
F15,NaN,"{'B4', 'B12', 'B7'}",0.0,#,15,B15
G16,='sheet1'!A6 + B12,"{'B6', 'B8'}",1.0,$,2428,B5
G17,=B5+B16+B14,nan,2.0,$,2491,B8
H16,=SUM(B18:B5),{'B9'},2.0,$,2508,B6
H17,=B6+B4+B3,nan,3.0,$,2596,B9


In [18]:
df1.df.loc['G16']

fml           ='sheet1'!A6 + B12
dependents              {B6, B8}
res_order                      1
ftype                          $
value                       2438
code                          B5
Name: G16, dtype: object

In [19]:
df2.df.loc['G16']

fml           ='sheet1'!A6 + B12
dependents              {B6, B8}
res_order                      1
ftype                          $
value                       2428
code                          B5
Name: G16, dtype: object

In [98]:
(
    pd.DataFrame(
    index=pd.Index([], name='cell', dtype=str),
    columns=TABLE_DATA_MAP.keys()
    )
    .astype(TABLE_DATA_MAP)
)


,fml,dependents,res_order,ftype,value,code
cell,,,,,,


In [110]:
from typing import Dict, List, TypedDict
import pandas as pd

# Define a TypedDict for the column names and their corresponding data types
class ColumnSpec(TypedDict):
    column_name: type

# Define a TypedDict for the structure of the DataFrame
class DataFrameStructure(TypedDict):
    index: List[type]  # List of types for the index
    columns: Dict[str, ColumnSpec]  # Dictionary of column names and their types

# Example structure definition
structure: DataFrameStructure = {
    'index': [int],  # Example: Index should be integers
    'columns': {
        'column1': int,
        'column2': str,
        'column3': float
    }
}


In [112]:
ColumnSpec

__main__.ColumnSpec

In [103]:

# Create a function to create DataFrame with specified structure
def create_dataframe(structure: DataFrameStructure) -> pd.DataFrame:
    # Create empty DataFrame with specified index and columns
    df = pd.DataFrame(index=pd.Index([], dtype=structure['index'][0]),
                      columns=structure['columns'].keys())
    
    # Set column types
    for col, col_type in structure['columns'].items():
        df[col] = df[col].astype(col_type)
    
    return df

# Example usage:
df = create_dataframe(structure)
print(df.dtypes)  # Print data types of the DataFrame


column1      int64
column2     object
column3    float64
dtype: object


In [109]:
df.index

Index([], dtype='int64')

In [83]:
S

,name
email,
bingley@xyz.com,Bingley
bennet@xyz.com,Elizabeth


In [84]:
df1 == df2

False

In [78]:

df1 & DfComparator(d)

,name
email,
